In [13]:
import json, re, sys
import pandas as pd
import numpy as np

from Bio.KEGG import REST


In [33]:
genome_file = "cazy_genomes_2021_05_15.json"
#genome_file = "test.json"

desired_rank = ["superkingdom", "phylum", "class", "order", "family", "genus", "species", "genome"]

taxon_dict = {}

parent_son = set()

with open("taxon.csv", "w") as output:
    output.write(",".join(["name", "taxid", "rank"]) + "\n")

with open("connections.csv", "w") as output:
    output.write(",".join(["from", "to"]) + "\n")

with open("has_cazy.csv", "w") as output:
    output.write(",".join(["from", "to", "amount"]) + "\n")

for line in open(genome_file, 'r'):
    genome = json.loads(line)

    if "superkingdom" in genome["taxonomy"] and genome["taxonomy"]["superkingdom"][0] in ["Bacteria", "Archaea"]:
        taxid = genome["taxid"]
        name = genome["name"]

        taxonomy = genome["taxonomy"]

        taxonomy["genome"] = [name, genome["taxid"]]

        quartett = [""] * 2
        for rank in desired_rank:
            if rank in taxonomy:
                taxon, taxon_taxid = taxonomy[rank]

                if taxon not in taxon_dict:
                    taxon_dict[taxon] = [taxon_taxid, rank]

                if rank == "superkingdom":
                    quartett[0] = taxon
                else:
                    quartett[1] = taxon

                    if quartett[0] != quartett[1]:
                        parent_son.add(tuple(quartett))

                        quartett[0] = quartett[1]
        
        taxon_dict[name] = [taxid, "genome"]

        for cazy in genome["cazy"]:
            with open("has_cazy.csv", "a") as output:
            #["~id", "~from", "~to", "~label"]
                output.write(",".join([f'"{name}"', f'"{cazy}"', f'{genome["cazy"][cazy]}']) + "\n")


        #print (genome)
#print (taxon_dict)


for name in taxon_dict:
    with open("taxon.csv", "a") as output:
        output.write(",".join([f'"{name}"', f'"{taxon_dict[name][0]}"', f'"{taxon_dict[name][1]}"']) + "\n")

for parent in parent_son:
    with open("connections.csv", "a") as output:
        #["~id", "~from", "~to", "~label"]
        output.write(",".join([f'"{parent[0]}"', f'"{parent[1]}"']) + "\n")

In [24]:
info_file = "cazy_info_2021_05_12.json"

rx_ec = re.compile(r'\b\d+\.\d+\.\d+\.\d+\b')

with open("cazy.csv", "w") as output:
    output.write(",".join(["name", "activities", "clan", "mechanism", "catalytic"]) + "\n")

with open("ec.csv", "w") as output:
    output.write(",".join(["name", "sysname", "reaction"]) + "\n")

with open("cazy_ec.csv", "w") as output:
    output.write(",".join(["from", "to"]) + "\n")

unique_ec = set()

for line in open(info_file, 'r'):
    cazy_line = json.loads(line)

    cazy = cazy_line["name"]
    activities = ""
    distribution = ""
    clan = ""
    mechanism = ""
    catalytic = ""

    if " Activities in Family" in cazy_line:
        activities = cazy_line[" Activities in Family"]

    if "distribution" in cazy_line:
        distribution = json.dumps(cazy_line["distribution"])

    if "Clan" in cazy_line:
        clan = cazy_line["Clan"]
    
    if "Mechanism" in cazy_line:
        mechanism = cazy_line["Mechanism"]
    
    if "Catalytic Nucleophile/Base" in cazy_line:
        catalytic = cazy_line["Catalytic Nucleophile/Base"]

    if "ec" in cazy_line:
        ecs = cazy_line["ec"]

        for ec in ecs:
            match_ec = rx_ec.match(ec)

            if match_ec:
                with open("cazy_ec.csv", "a") as output:
                    output.write(",".join([f'"{cazy}"', f'"{ec}"']) + "\n")
                
                unique_ec.add(ec)
        
        with open("cazy.csv", "a") as output:
            output.write(",".join([f'"{cazy}"', f'"{activities}"', f'"{clan}"', f'"{mechanism}"', f'"{catalytic}"']) + "\n")

for ec in unique_ec:
    with open("ec.csv", "a") as output:
        output.write(",".join([f'"{ec}"']) + "\n")

In [21]:
with open("ec_final.csv", "w") as output:
    output.write(",".join(["name", "sysname", "reaction"]) + "\n")

is_header = True
for line in open("ec.csv"):

    if is_header == True:
        is_header = False
    else:

        ec = f"ec:{line.strip()}"
        sysname = ""
        reaction = ""
        for content in REST.kegg_get(ec):
            for l in content.split("\n"):
                if l.startswith("SYSNAME"):
                    sysname = l.replace("SYSNAME", "").strip()
            
                if l.startswith("REACTION"):
                    reaction = l.replace("REACTION", "").strip()
        
        #print (f'{line.strip()},"{sysname}","{reaction}"')
        with open("ec_final.csv", "a") as output:
            output.write(",".join([line.strip(), f'"{sysname}"', f'"{reaction}"']) + "\n")